# Extracting Historical Data using Kite's Historical API for AlgoTrading

### Import necessary modules

In [1]:
from selenium import webdriver
from kiteconnect import KiteConnect

import time
import pandas as pd
import datetime as dt

### Store your api_key, api_secret & chromedriver path in a variable

In [ ]:
api_key    = '<YOUR API KEY>'
api_secret = '<YOUR API SECRET>'
chromedriver_path = '/Users/algotrade/Desktop'

### Launch Chrome using Selenium webdriver

In [25]:
kite = KiteConnect(api_key = api_key)

service = webdriver.chrome.service.Service(f'{chromedriver_path}//chromedriver.exe')
service.start()

options = webdriver.ChromeOptions()

options = options.to_capabilities()
driver = webdriver.Remote(service.service_url, options)

driver.get(kite.login_url())

C:\Users\ganesh\AppData\Local\Temp\ipykernel_13992\1676134305.py:9: DeprecationWarning: desired_capabilities has been deprecated, please pass in an Options object with options kwarg
  driver = webdriver.Remote(service.service_url, options)


- After successfully running above code, you will be redirected to login page.
- Login using your user id & password

### Request token

In [26]:
driver.current_url

'https://127.0.0.1/?action=login&type=login&status=success&request_token=IgabELA03crEdvKUV9vHRu5TWp3qe5kx'

In [27]:
request_token = 'IgabELA03crEdvKUV9vHRu5TWp3qe5kx'

### Access token

In [28]:
access_token = kite.generate_session(request_token, api_secret = api_secret)["access_token"]
access_token

'2U0ncZR0nhxi64t6vaAjqHnlCVBLjKNi'

### Create NSE/BSE dataframe to fetch instrument token number

In [29]:
nse = kite.instruments('NSE')
nse_data = pd.DataFrame(nse)
nse_data

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,264713,1034,HANGSENG BEES-NAV,HANGSENG BEES-NAV,0.0,,0.0,0.00,0,EQ,INDICES,NSE
1,264969,1035,INDIA VIX,INDIA VIX,0.0,,0.0,0.00,0,EQ,INDICES,NSE
2,260617,1018,NIFTY 100,NIFTY 100,0.0,,0.0,0.00,0,EQ,INDICES,NSE
3,264457,1033,NIFTY 200,NIFTY 200,0.0,,0.0,0.00,0,EQ,INDICES,NSE
4,256265,1001,NIFTY 50,NIFTY 50,0.0,,0.0,0.00,0,EQ,INDICES,NSE
...,...,...,...,...,...,...,...,...,...,...,...,...
4521,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,,0.0,0.05,1,EQ,NSE,NSE
4522,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,,0.0,0.05,1,EQ,NSE,NSE
4523,979713,3827,ZUARIIND,ZUARI INDUSTRIES,0.0,,0.0,0.05,1,EQ,NSE,NSE
4524,2029825,7929,ZYDUSLIFE,ZYDUS LIFESCIENCES,0.0,,0.0,0.05,1,EQ,NSE,NSE


### Function to fetch instrument token number

In [30]:
def instrument_token(data, symbol):
    """
    This function will return the token number of the instrument from data
    
    """
    token_number = data[data.tradingsymbol == symbol].instrument_token.values[0]
    
    return token_number

In [31]:
#getting instrument token number for NIFTY 50

inst_token = instrument_token(nse_data, 'NIFTY 50')
inst_token

256265

In [36]:
nifty_50_data = pd.DataFrame(kite.historical_data(inst_token, '2021-01-01' , '2021-02-28', 'minute'))
nifty_50_data

,date,open,high,low,close,volume
0,2021-01-01 09:15:00+05:30,13996.10,14019.50,13994.85,14013.15,0
1,2021-01-01 09:16:00+05:30,14014.85,14018.55,14008.15,14009.05,0
2,2021-01-01 09:17:00+05:30,14008.05,14013.10,14005.05,14012.70,0
3,2021-01-01 09:18:00+05:30,14013.65,14019.10,14013.65,14016.20,0
4,2021-01-01 09:19:00+05:30,14015.45,14017.80,14011.95,14015.45,0
...,...,...,...,...,...,...
14749,2021-02-26 15:25:00+05:30,14477.70,14482.25,14476.70,14476.90,0
14750,2021-02-26 15:26:00+05:30,14479.00,14485.50,14477.70,14485.10,0
14751,2021-02-26 15:27:00+05:30,14485.00,14503.05,14485.00,14502.20,0
14752,2021-02-26 15:28:00+05:30,14505.60,14511.50,14502.00,14504.65,0


In [38]:
pd.DataFrame(kite.historical_data(inst_token, '2021-01-01' , '2021-05-28', 'minute'))

InputException: interval exceeds limit: 60 days

#### Error will be raised for above code because, the time interval in greater than the default limit. Limits for various intervals are:

- Minute : 60 days
- 3minute : 100 days
- 5minute : 100 days
- 10minute : 100 days
- 15minute : 200 days
- 30minute : 200 days
- 60minute : 400 days
- 1 Day : 2000 days

### Function which will return historical data irrespective of their time limits

In [40]:
def historical_data(symbol, from_date, to_date, interval):
    """
    This function will return historical data of the instrument for specific period of days for specific interval
    
    """
    
    df = pd.DataFrame()   
    int_token = instrument_token(nse_data, symbol)
    
    to_date   = pd.Timestamp(to_date)
    from_date = pd.Timestamp(from_date)

    while True:
        if from_date >= (to_date - dt.timedelta(60)):            #if from_date is within the 60 days limit
            df = df.append(pd.DataFrame(kite.historical_data(int_token, from_date, to_date, interval)))
            break
            
        else:                                                    #if from_date has more than 60 days limit
            to_date_new = from_date + dt.timedelta(60)
            
            df = df.append(pd.DataFrame(kite.historical_data(int_token, from_date, to_date_new, interval)))
            
            #to_date = from_date.date() + dt.timedelta(60)
            from_date = to_date_new
            
    return df

In [42]:
# fetching per minute historical data for NIFTY 50 for whole year of 2020

NIFTY_50_2020 = historical_data('NIFTY 50', '01-01-2020', '31-12-2020', 'minute')
NIFTY_50_2020

C:\Users\ganesh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3280: UserWarning: Parsing '31-12-2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  if await self.run_code(code, result, async_=asy):
C:\Users\ganesh\AppData\Local\Temp\ipykernel_13992\2196610737.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(kite.historical_data(int_token, from_date, to_date_new, interval)))
C:\Users\ganesh\AppData\Local\Temp\ipykernel_13992\2196610737.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(kite.historical_data(int_token, from_date, to_date_new, interval)))
C:\Users\ganesh\AppData\Local\Temp\ipy

,date,open,high,low,close,volume
0,2020-01-01 09:15:00+05:30,12202.15,12215.90,12202.15,12215.10,0
1,2020-01-01 09:16:00+05:30,12214.55,12216.15,12205.70,12206.80,0
2,2020-01-01 09:17:00+05:30,12205.20,12207.40,12203.15,12203.55,0
3,2020-01-01 09:18:00+05:30,12202.15,12208.40,12201.45,12208.35,0
4,2020-01-01 09:19:00+05:30,12208.50,12213.10,12207.70,12211.30,0
...,...,...,...,...,...,...
1120,2020-12-30 15:25:00+05:30,13987.60,13989.60,13985.85,13986.05,0
1121,2020-12-30 15:26:00+05:30,13985.90,13987.25,13983.60,13985.50,0
1122,2020-12-30 15:27:00+05:30,13984.70,13985.65,13981.35,13982.45,0
1123,2020-12-30 15:28:00+05:30,13981.95,13983.30,13972.75,13973.20,0
